In [1]:
import copy
import pandas as pd
import numpy as np

EXCEL_FILE = "datasets/hdruk_researchfish_2020.xlsx"
DATA = []

In [2]:
# Reading in Excel Sheets
awards = pd.read_excel(EXCEL_FILE, sheet_name="1A Awards").rename(columns={"Award Reference": "id"}).set_index('id').dropna(axis=1).convert_dtypes()
pubs = pd.read_excel(EXCEL_FILE, sheet_name="Pub V2").rename(columns={"Publication ID": "id"}).set_index('id').dropna(axis=1).convert_dtypes()
collabs = pd.read_excel(EXCEL_FILE, sheet_name="1C Collabs").set_index('id').convert_dtypes()
further_funding = pd.read_excel(EXCEL_FILE, sheet_name="1D Further Funding").set_index('id').dropna(axis=1).convert_dtypes()
next_destination = pd.read_excel(EXCEL_FILE, sheet_name="1E Next Destination").set_index('id').dropna(axis=1).convert_dtypes()
skills_shortage = pd.read_excel(EXCEL_FILE, sheet_name="1F Skills Shortage").rename(columns={"ID": "id"}).dropna(axis=1).convert_dtypes()
secondments = pd.read_excel(EXCEL_FILE, sheet_name="1F Secondments").set_index('id').dropna(axis=1).convert_dtypes()
engagements = pd.read_excel(EXCEL_FILE, sheet_name="1H Engagements").set_index('id').dropna(axis=1).convert_dtypes()
policies = pd.read_excel(EXCEL_FILE, sheet_name="1I Policy").set_index('id').dropna(axis=1).convert_dtypes()
tools = pd.read_excel(EXCEL_FILE, sheet_name="1J Tools").set_index('id').dropna(axis=1).convert_dtypes()
databases = pd.read_excel(EXCEL_FILE, sheet_name="1K Databases").set_index('id').dropna(axis=1).convert_dtypes()
software = pd.read_excel(EXCEL_FILE, sheet_name="1L Software").set_index('id').dropna(axis=1).convert_dtypes()
artistic = pd.read_excel(EXCEL_FILE, sheet_name="1M Artistic Creative").set_index('id').dropna(axis=1).convert_dtypes()
products = pd.read_excel(EXCEL_FILE, sheet_name="1O Products").set_index('id').dropna(axis=1).convert_dtypes()
recognition = pd.read_excel(EXCEL_FILE, sheet_name="1R Awards Recognition").set_index('id').dropna(axis=1).convert_dtypes()
facilities = pd.read_excel(EXCEL_FILE, sheet_name="1S Facilities").set_index('id').dropna(axis=1).convert_dtypes()
other = pd.read_excel(EXCEL_FILE, sheet_name="1T Other").set_index('id').dropna(axis=1).convert_dtypes()
animal_use = pd.read_excel(EXCEL_FILE, sheet_name="1U Animal Use").rename(columns={"ID": "id"}).set_index('id').dropna(axis=1).convert_dtypes()


In [56]:
# Map Priority Areas
def priority_area(row):
    PRIORITY_AREA_MAP = {
        "central infrastructure activities": "Infrastructure",
        "central PPPEI activities": "PPIE",
        "central training activities": "Training",
        "Applied Analytics": "Applied Analytics",
        "Better Care": "Better Care",
        "Human Phenome": "Human Phenome",
        "Understanding Causes of Disease": "Aetiology of Disease",
        "Clinical Trials": "Clinical Trials",
        "Improving Public Health": "Improving Public Health",
        "ISCF HDRUK DIH Sprint Exemplar": "Sprint Exemplars",
        "Innovation Gateway" : "Infrastructure",
        "Data and Connectivity": "Data and Connectivity",

    }
    for k in list(PRIORITY_AREA_MAP.keys()):
        if k in row['Award Title']:
            return PRIORITY_AREA_MAP[k]
    return "Fellowship"


awards['priority area'] = awards.apply(lambda row: priority_area(row), axis=1)

array(['Infrastructure', 'PPIE', 'Training', 'Applied Analytics',
       'Better Care', 'Human Phenome', 'Aetiology of Disease',
       'Clinical Trials', 'Improving Public Health', 'Sprint Exemplars',
       'Data and Connectivity', 'Fellowship'], dtype=object)

In [132]:
# Join dataframes
pubs_merged = pd.merge(pubs, awards, left_on="Award Reference", right_on="id", how='left')
projects_merged = pd.merge(collabs, awards, left_on="Award Reference", right_on="id", how='left')
tools_merged = pd.merge(tools, awards, left_on="Award Reference", right_on="id", how='left')
databases_merged = pd.merge(databases, awards, left_on="Award Reference", right_on="id", how='left')
software_merged = pd.merge(software, awards, left_on="Award Reference", right_on="id", how='left')

In [133]:
# Collect collaborations
COLLABORATIONS = {}
for index, p in pubs_merged.iterrows():
    COLLABORATIONS.setdefault(p['Outcome Base ID'],[]).append(p['priority area'])

for index, p in projects_merged.iterrows():
    COLLABORATIONS.setdefault(p['collaboration Base ID'],[]).append(p['priority area'])

for index, p in tools_merged.iterrows():
    COLLABORATIONS.setdefault(p['Outcome Base ID'],[]).append(p['priority area'])

for index, p in databases_merged.iterrows():
    COLLABORATIONS.setdefault(p['Outcome Base ID'],[]).append(p['priority area'])

for index, p in software_merged.iterrows():
    COLLABORATIONS.setdefault(p['Outcome Base ID'],[]).append(p['priority area'])

In [134]:
# Build Collaborations map
COLLABORATION_MAP = {}
for _, values in COLLABORATIONS.items():
    values = set(values)
    for pa in values:
        COLLABORATION_MAP.setdefault(pa, {}).setdefault(pa,0)
        COLLABORATION_MAP[pa][pa] += 1
        remaining = copy.deepcopy(values)
        remaining.remove(pa)
        for remaining_pa in remaining:
            COLLABORATION_MAP[pa].setdefault(remaining_pa,0)
            COLLABORATION_MAP[pa][remaining_pa] += 1

In [135]:
# Filter out PRIORITY AREAS
# PRIORITY_AREAS = awards['priority area'].unique().tolist()

PRIORITY_AREAS = [
#  'PPIE',
 'Applied Analytics',
#  "Infrastructure",
 'Better Care',
 'Data and Connectivity',
 'Human Phenome',
  'Sprint Exemplars',
#  'Training',
 'Fellowship',
 'Aetiology of Disease',
 'Improving Public Health',
 'Clinical Trials',
]

# Generate collaboration Matrix
COLLABORATION_MATRIX = []
for source_pa in PRIORITY_AREAS:
    matrix = []
    for target_pa in PRIORITY_AREAS:
        matrix.append(COLLABORATION_MAP.get(source_pa, {}).get(target_pa, 0))
    COLLABORATION_MATRIX.append(matrix)
COLLABORATION_MATRIX

[[194, 83, 0, 138, 0, 0, 147, 84, 12],
 [83, 488, 0, 223, 0, 20, 152, 186, 15],
 [0, 0, 76, 0, 0, 0, 0, 0, 0],
 [138, 223, 0, 649, 7, 45, 180, 270, 11],
 [0, 0, 0, 7, 47, 9, 0, 0, 16],
 [0, 20, 0, 45, 9, 559, 13, 17, 0],
 [147, 152, 0, 180, 0, 13, 501, 98, 13],
 [84, 186, 0, 270, 0, 17, 98, 601, 11],
 [12, 15, 0, 11, 16, 0, 13, 11, 49]]

In [136]:
# Build Chort Diagram
from chord import Chord

c = Chord(COLLABORATION_MATRIX, PRIORITY_AREAS,  width=800,
        # margin=200,
        # colors="d3.schemeSet2",
        # colors=f"d3.schemeGnBu[{len(PRIORITY_AREAS)}]",
        # colors="d3.schemeSet2",
        #   curved_labels=True,
        arc_numbers=True,
        # opacity=0.5,
        font_size="5px", font_size_large="12px",
        # divide=True, divide_idx=4,
        conjunction="&", verb="collaborate in", noun="projects",
        title="National Priority Area Collaborations"
    )
c.to_png()
c.to_html()